<h1> MJRL training </h1>

In [2]:
import warnings
warnings.filterwarnings('ignore')

import gym
from mjrl.utils.gym_env import GymEnv
from mjrl.policies.gaussian_mlp import MLP
from mjrl.baselines.mlp_baseline import MLPBaseline
from mjrl.algos.ppo_clip import PPO
from mjrl.algos.npg_cg import NPG
from mjrl.utils.train_agent import train_agent
import myosuite

# Create the environment
env = gym.make('CenterReachOut-v0')
env.reset()


MyoSuite:> Registering Myo Envs
    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    


array([-2.2856e-02,  1.1130e-02, -8.2680e-02,  1.0125e-01, -4.9400e-02,
        1.6578e-01,  1.1868e-01, -1.9320e-01, -1.9648e-01,  3.8760e-02,
        1.6806e-01,  1.7281e-01, -2.5407e-01, -1.0998e-01,  1.4748e+00,
        1.2882e+00,  1.0647e-01, -2.7489e-01,  3.3200e-01,  4.1140e-01,
       -1.7716e-01, -2.6182e-01,  1.8852e-01,  1.7017e-01,  4.4773e-01,
        2.6707e-01,  2.8278e-01, -1.0472e-02,  1.6854e-02,  7.0198e-03,
        9.1390e-02, -6.5450e-02,  2.4350e-01,  3.3776e-01,  2.3565e-01,
       -1.0184e-01,  6.0707e-04,  3.1420e-02, -2.0000e-01,  0.0000e+00,
       -2.0000e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
        0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.00

<h1> Hyperparameter tuning </h1>

In [3]:
from sklearn.model_selection import RandomizedSearchCV



In [4]:
#Policy hyperparameters
policy_size = (256, 256)  # Size of the policy network
seed = 123  # Random seed used for reproducibility
rl_step_size = 0.1
learnrate = 0.025
entcoeff = 0.01
batchsize = 32
num_epoch = 10

#Value Function hyperparameters
vf_hidden_size = (256, 256) # Value Function Network size
seed = 123  # Random seed used for reproducibility
rl_step_size = 0.1
learnrate = 0.025
entcoeff = 0.01
batchsize = 32
num_epoch = 10
regcoef = 1e-3

#Training hyperparameters
iterations = 1000
gamma = 0.995
gae_lambda = 0.97


# Wrap the environment
e = GymEnv(env)

<H1>Set Policies </h1>

In [5]:
# Initialize policy and baseline
policy = MLP(e.spec, hidden_sizes=policy_size, seed=seed, init_log_std=-0.25, min_log_std=-1.0)
baseline = MLPBaseline(e.spec, reg_coef=regcoef, batch_size=batchsize , hidden_sizes=vf_hidden_size, epochs=num_epoch, learn_rate=learnrate)

# Initialize PPO agent
agent = NPG(e, policy, baseline, normalized_step_size=rl_step_size, seed=seed, save_logs=True,ent_coeff=entcoeff ,tensorboard_log="./ppo_objhold_tensorboard/")


<h1> Start Training </h1>

In [6]:
print("========================================")
print("Starting policy learning")
print("========================================")

# Train the agent
train_agent(job_name='.',
            agent=agent,
            seed=seed,
            niter=10000,
            gamma=gamma,
            gae_lambda=gae_lambda,
            sample_mode="trajectories",
            num_traj=96,
            num_samples=0,
            save_freq=100,
            evaluation_rollouts=10)

print("========================================")
print("Job Finished.")
print("========================================")

Starting policy learning
Reading: logs\log.csv
Log read from logs\log.csv: had 2000 entries
Loaded last saved iteration: 1900
Resuming from an existing job folder ...
......................................................................................
ITERATION : 1901 
Performing evaluation rollouts ........
[ Wed Apr  3 01:30:13 2024 ] 1901 -515.96 -150.02 -100000000.00 
------------------  -------------
VF_error_after          0.0050484
VF_error_before         0.233937
alpha                   0.613937
delta                   0.1
env_samples          2518
eval_score           -150.02
eval_success            0
iteration            1900
kl_dist                 0.050148
num_samples          2518
rollout_success         0
running_score        -515.961
rwd_dense              -5.16568
rwd_sparse             -0.147099
stoc_pol_max          288.424
stoc_pol_mean        -515.961
stoc_pol_min        -4855.71
stoc_pol_std         1414.16
success_percentage      0
surr_improvement        0.1262

<h1> Logging hyperparameters

In [ ]:
# Data to be written into the file
data = f"""
# Policy hyperparameters
policy_size = {policy_size}
seed = {seed}
rl_step_size = {rl_step_size}
learnrate = {learnrate}
entcoeff = {entcoeff}
batchsize = {batchsize}
num_epoch = {num_epoch}

# Value Function hyperparameters
vf_hidden_size = {vf_hidden_size}
seed = {seed}
rl_step_size = {rl_step_size}
learnrate = {learnrate}
entcoeff = {entcoeff}
batchsize = {batchsize}
num_epoch = {num_epoch}
regcoef = {regcoef}


"""

# Write data to the file
with open("myoarmreach#9/readme.txt", "w") as file: #Change the folder name 
    file.write(data)

FileNotFoundError: [Errno 2] No such file or directory: 'myoarmreach#9/readme.txt'